# Novel Approach and Analysis Techniques

In [1]:
pip install torch pytorch-lightning pytorch_forecasting

     ------------------------------------ 141.4/141.4 kB 270.9 kB/s eta 0:00:00
     --------------------------------------- 42.5/42.5 MB 75.8 kB/s eta 0:00:00
     ------------------------------------ 308.2/308.2 kB 130.6 kB/s eta 0:00:00
     ------------------------------------ 210.5/210.5 kB 188.4 kB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 210.6 kB/s eta 0:00:00
     --------------------------------------- 81.0/81.0 kB 79.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ------------------------------------- 192.2/192.2 kB 79.8 kB/s eta 0:00:00
     --------------------------------------- 78.7/78.7 kB 95.3 kB/s eta 0:00:00
     ------------------------------------ 147.2/147.2 kB 121.8 kB/s eta 0:00:00
     -------------------------------------- 49.6/49.6 kB 114.8 kB/s eta 0:00:00
     -------------------------------------- 95.2/95.2 kB 118.2 kB/s eta 0:00:00
  Using cached pyperclip-1.8.2.tar.gz (20 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 112.7/112.7 kB 198.9 kB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=a66f086de2867dcacaed6dc58724f0ce066c02d21c9d4947bd4ab54676cc57b2
  Stored in directory: c:\users\tamojit\appdata\local\pip\cache\wheels\3c\77\81\aaa2802e9b0553585f2789c6f2756b50a09a01d2848423bb15
Successfully built pyperclip
  Attempting uninstall: scipy
    Found existing install

In [3]:
import numpy as np
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet

In [6]:
data = pd.read_csv('divvy-tripdata_cleaned.csv', index_col=1)
data.index = pd.to_datetime(data.index)
data.sort_index(inplace=True)
data.head(5)

,trip_id,stop_time,bike_id,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,gender,...,from_longitude,from_location,to_latitude,to_longitude,to_location,date,year,month,day,day_of_week
start_time,,,,,,,,,,,,,,,,,,,,,
2013-06-30 10:48:00,10009,06/30/2013 10:58:00 AM,354,557,58,Ashland Ave & Armitage Ave,60,Dayton St & North Ave,Subscriber,Female,...,-87.668879,POINT (-87.668879 41.916017),41.910578,-87.649422,POINT (-87.649422 41.910578),2013-06-30,2013,6,30,Sunday
2013-06-30 10:51:00,10017,06/30/2013 02:19:00 PM,559,12496,37,Dearborn St & Adams St,34,Cannon Dr & Fullerton Ave,Customer,NaN,...,-87.629791,POINT (-87.629791 41.879356),41.926756,-87.634429,POINT (-87.634429 41.926756),2013-06-30,2013,6,30,Sunday
2013-06-30 11:27:00,10106,06/30/2013 11:39:00 AM,616,699,17,Wood St & Division St,69,Damen Ave & Pierce Ave,Customer,NaN,...,-87.672730,POINT (-87.67273 41.90332),41.909396,-87.677692,POINT (-87.677692 41.909396),2013-06-30,2013,6,30,Sunday
2013-06-30 11:35:00,10133,06/30/2013 11:53:00 AM,136,1093,127,Lincoln Ave & Fullerton Ave,26,McClurg Ct & Illinois St,Customer,NaN,...,-87.649260,POINT (-87.64926 41.925905),41.891020,-87.617300,POINT (-87.6173 41.89102),2013-06-30,2013,6,30,Sunday
2013-06-30 11:56:00,10189,06/30/2013 01:02:00 PM,279,3957,72,State St & 16th St,15,Racine Ave & 18th St,Customer,NaN,...,-87.625813,POINT (-87.625813 41.860384),41.858166,-87.656495,POINT (-87.656495 41.858166),2013-06-30,2013,6,30,Sunday


In [10]:
data = data.resample('1h').mean().replace(0., np.nan)
earliest_time = data.index.min()
df=data[['trip_id','bike_id','trip_duration','from_station_id','to_station_id']]

In [11]:
df_list = []

for label in df:

    ts = df[label]

    start_date = min(ts.fillna(method='ffill').dropna().index)
    end_date = max(ts.fillna(method='bfill').dropna().index)

    active_range = (ts.index >= start_date) & (ts.index <= end_date)
    ts = ts[active_range].fillna(0.)

    tmp = pd.DataFrame({'trip_duration': ts})
    date = tmp.index

    tmp['hours_from_start'] = (date - earliest_time).seconds / 60 / 60 + (date - earliest_time).days * 24
    tmp['hours_from_start'] = tmp['hours_from_start'].astype('int')
  
    tmp['days_from_start'] = (date - earliest_time).days
    tmp['date'] = date
    tmp['bikeshare_id'] = label
    tmp['hour'] = date.hour
    tmp['day'] = date.day
    tmp['day_of_week'] = date.dayofweek
    tmp['month'] = date.month

    # stack all time series vertically
    df_list.append(tmp)

time_df = pd.concat(df_list).reset_index(drop=True)

# match results in the original paper
time_df = time_df[(time_df['days_from_start'] >= 1096)
                & (time_df['days_from_start'] < 1346)].copy()

In [1]:
time_df[['bikeshare_id','trip_duration']].groupby('bikeshare_id').mean()

In [ ]:
# hyperparameters
# batch size=64
# number heads=4, hidden sizes=160, lr=0.001, gr_clip=0.1

max_prediction_length = 24
max_encoder_length = 7*24
training_cutoff = time_df["hours_from_start"].max() - max_prediction_length

training = TimeSeriesDataSet(
    time_df[lambda x: x.hours_from_start <= training_cutoff],
    time_idx="hours_from_start",
    target="trip_duration",
    group_ids=["bikeshare_id"],
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["bikeshare_id"],
    time_varying_known_reals=["hours_from_start","day","day_of_week", "month", 'hour'],
    time_varying_unknown_reals=['trip_duration'],
    target_normalizer=GroupNormalizer(
        groups=["bikeshare_id"], transformation="softplus"
    ),  # we normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)


validation = TimeSeriesDataSet.from_dataset(training, time_df, predict=True, stop_randomization=True)

# create dataloaders for  our model
batch_size = 64 
# if you have a strong GPU, feel free to increase the number of workers  
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [ ]:
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5, verbose=True, mode="min")
lr_logger = LearningRateMonitor()  
logger = TensorBoardLogger("lightning_logs")  

trainer = pl.Trainer(
    max_epochs=45,
    accelerator='gpu', 
    devices=1,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    callbacks=[lr_logger, early_stop_callback],
    logger=logger)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.001,
    hidden_size=160,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=160,
    output_size=7,  # there are 7 quantiles by default: [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]
    loss=QuantileLoss(),
    log_interval=10, 
    reduce_on_plateau_patience=4)

trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader)

In [ ]:
best_model_path = trainer.checkpoint_callback.best_model_path
print(best_model_path)
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# load our saved model again
!unzip model.zip
best_model_path='lightning_logs/lightning_logs/version_1/checkpoints/epoch=8-step=4212.ckpt'
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# start tensorboard
%load_ext tensorboard
%tensorboard - logdir lightning_logs

In [ ]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)

# average p50 loss overall
print((actuals - predictions).abs().mean().item())
# average p50 loss per time series
print((actuals - predictions).abs().mean(axis=1))

In [ ]:
# take a look at what the raw_predictions variable contains

raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
print(raw_predictions._fields)
# ('prediction', 
# 'encoder_attention', 
# 'decoder_attention', 
# 'static_variables', 
# 'encoder_variables', 
# 'decoder_variables', 
# 'decoder_lengths', 
# 'encoder_lengths')

print('\n')
print(raw_predictions['prediction'].shape)
# torch.Size([5, 24, 7])

# we get predictions of 5 time-series for 24 days.
# for each day we get 7 predictions - these are the 7 quantiles:
# [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]
# we are mostly interested in the 4th quantile which represents, let's say, the 'median loss'
# fyi, although docs use the term quantiles, the most accurate term are percentiles

In [ ]:
for idx in range(5):  # plot all 5 consumers
    fig, ax = plt.subplots(figsize=(10, 4))
    best_tft.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True,ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

raw_prediction, x = best_tft.predict(
    training.filter(lambda x: (x.consumer_id == "MT_004") & (x.time_idx_first_prediction == 26512)),
    mode="raw",
    return_x=True,
)
best_tft.plot_prediction(x, raw_prediction, idx=0, ax=ax);

In [ ]:
# encoder data is the last lookback window: we get the last 1 week (168 datapoints) for all 5 consumers = 840 total datapoints

encoder_data = time_df[lambda x: x.hours_from_start > x.hours_from_start.max() - max_encoder_length]
last_data = time_df[lambda x: x.hours_from_start == x.hours_from_start.max()]

# decoder_data is the new dataframe for which we will create predictions. 
# decoder_data df should be max_prediction_length*consumers = 24*5=120 datapoints long : 24 datapoints for each cosnumer
# we create it by repeating the last hourly observation of every consumer 24 times since we do not really have new test data
# and later we fix the columns

decoder_data = pd.concat(
    [last_data.assign(date=lambda x: x.date + pd.offsets.Hour(i)) for i in range(1, max_prediction_length + 1)],
    ignore_index=True,
)

# fix the new columns
decoder_data["hours_from_start"] = (decoder_data["date"] - earliest_time).dt.seconds / 60 / 60 + (decoder_data["date"] - earliest_time).dt.days * 24
decoder_data['hours_from_start'] = decoder_data['hours_from_start'].astype('int')
decoder_data["hours_from_start"] += encoder_data["hours_from_start"].max() + 1 - decoder_data["hours_from_start"].min()

decoder_data["month"] = decoder_data["date"].dt.month.astype(np.int64)
decoder_data["hour"] = decoder_data["date"].dt.hour.astype(np.int64)
decoder_data["day"] = decoder_data["date"].dt.day.astype(np.int64)
decoder_data["day_of_week"] = decoder_data["date"].dt.dayofweek.astype(np.int64)

new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

fig, ax = plt.subplots(figsize=(10, 5))

# create out-of-sample predictions for MT_002
new_prediction_data=new_prediction_data.query(" consumer_id == 'MT_002'")
new_raw_predictions, new_x = best_tft.predict(new_prediction_data, mode="raw", return_x=True)
best_tft.plot_prediction(new_x, new_raw_predictions, idx=0, show_future_observed=False, ax=ax);

In [ ]:
raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
interpretation = best_tft.interpret_output(raw_predictions, reduction="sum")
best_tft.plot_interpretation(interpretation)

In [ ]:
# analysis on the training set

predictions, x = best_tft.predict(train_dataloader, return_x=True)
predictions_vs_actuals = best_tft.calculate_prediction_actual_by_variable(x, predictions)
best_tft.plot_prediction_actual_by_variable(predictions_vs_actuals);

In [ ]:
# create a new study
study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path="optuna_test",
    n_trials=1,
    max_epochs=1,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(30, 128),
    hidden_continuous_size_range=(30, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False 
)

# save study results
with open("test_study.pkl", "wb") as fout:
    pickle.dump(study, fout)

# print best hyperparameters
print(study.best_trial.params)